# Extracción de contenido con Python a partir de JSON

<html><p><span style="font-size:x-small">En GitHub, los enlaces de la tabla de contenidos de un notebook Jupyter no funcionan. Para una versión con enlaces, abrir en <a href="">NbViewer</a>. Para una versión interactiva del notebook, abrir en Binder:</span></p></html>

- [Introducción: JSON](#s-intro)
- [Extracción del contenido](#s-extraccion-xml)
  - [Metadatos](#ss-metadatos)
  - [Texto del poema](#ss-texto)
- [Guardar la información extraída en una dataframe](#s-guardar-dataframe)

<a name="s-intro"></a>

# Introducción: JSON

Como se vio en el capítulo, JSON (acrónimo de *JavaScript Object Notation*, y que podemos pronunciar como *jotasón*) es un lenguaje para representar contenido de forma estructurada. El formalismo para representar JSON mediante una cadena de texto identifica el contenido con pares anidables de clave-valor, en vez de con pares de etiquetas anidables, contrariamente al formalismo de los formatos HTML y XML.

Como se vio en el [presente repositorio](../01-extraccion-con-openrefine/02-extraccion-desde-json.md), OpenRefine tiene algunas funciones para manipular JSON. En este notebook abordamos la manipulación de JSON con el lenguaje Python.


<a name="ss-sintaxis"></a>

## Sintaxis

En JSON la información se estructura con pares de clave-valor.

En el ejemplo, vemos claves como *poema*, *texto* y otras.

Hay varios tipos de valores posibles, los que vemos en el ejemplo son:

- cadenas de caracteres: P. ej. el valor asociado a la clave *nombre* dentro de *autoria*
- objetos: Los valores que se dan entre llaves `{}`. P. ej. los valores asociados a la clave *metadatos* y a la clave *autoria* son objetos
- listas: Aparecen entre `[]`. El valor para la clave *versos* es una lista, cuyos miembros son las cadenas de texto de cada verso

En JSON también puede haber valores numéricos y booleanos (verdaero, falso), así como un valor nulo.

Ya que las comillas forman parte de la sintaxis (tanto las claves como los valores de tipo cadena van entre comillas), si queremos usar comillas dentro de un valor, irán escapadas con barra inversa `\`, p. ej. el título del poema es *frigo y "frigo"*, que en JSON es `"frigo & \"frigo\""`

<a name="frag-ejemplo-json"></a>

**Ejemplo de documento expresado en JSON**
<html><span style="font-size:small">Elaboración propia</span></html>

```json
{
  "poema": {
    "metadatos": {
      "autoria": {
        "nombre": "Guillermo Carlos",
        "apellido": "Guillermos"
      },
      "titulo": "frigo & \"frigo\"",
      "fecha": ""
    },
    "texto": {
      "titulo": "frigo & \"frigo\"",
      "estrofas": [
        {
          "estrofa": {
            "tipo": "haiku",
            "versos": [
              "come la fruta",
              "del refrigerador",
              "muy deliciosa"
            ]
          }
        }
      ]
    }
  }
}

```

Los datos en JSON, como pasa con los datos en XML, son datos ya estructurados: Hay identificadores (claves en JSON, etiquetas en XML) asociados a diferentes tipos de contenido, y podemos procesar el contenido en función de los identificadores. Cuando hacemos *scraping*, típicamente pasamos de contenido no estructurado (p. ej en HTML o en texto plano) a contenido estructurado. No obstante, puede haber contenido en JSON como parte de los contenidos web que manipulamos con el *scraping*, con lo cual se menciona el formato aquí. 


<a name="s-extraccion-xml"></a>

# Extracción del contenido

Wikisource también puede ofrecer su contenido en JSON, a través de comandos de su API (Application Programming Interface o interfaz de programación de aplicaciones, es decir, conjunto de instrucciones para interaccionar con un programa a partir de otro programa).

Bajaremos con la API la página del poema ya visto en el capítulo, "Una viñeta" de Delmira Agustini, pero en JSON en vez de en HTML. El URL es https://es.wikisource.org/w/api.php?action=parse&page=Una_vi%C3%B1eta&format=json.

No se dan detalles sobre la parte del código que ya se ha visto en el [notebook](01-extraccion-desde-html.ipynb) sobre scraping desde HTML.

La novedad aquí es que, en vez de crear un árbol con BeautifulSoup, creamos un objeto JSON, asignado a la variable `ojson` y extraemos el contenido que nos interesa usando métodos (funciones) asociados al objeto. Python tiene un módulo por defecto para manipular JSON, llamado `json`, que se importa aquí. Para crear un objeto JSON desde una cadena de caracteres, se usa la instrucción `loads()` (la letra *s* en *loads* nos recuerda que estamos cargando una cadena o *string*).

In [1]:
import json
import requests

url_pagina = "https://es.wikisource.org/w/api.php?action=parse&page=Una_vi%C3%B1eta&format=json"
respuesta = requests.get(url_pagina)
contenido = respuesta.text
ojson =  json.loads(contenido)

In [2]:
ojson

{'parse': {'title': 'Una viñeta',
  'pageid': 27121,
  'revid': 1193682,
  'text': {'*': '<div class="mw-parser-output"><style data-mw-deduplicate="TemplateStyles:r1188054">.mw-parser-output #headertemplate{text-align:center;padding:1em;background:#F4F0E5;border-top:1px solid #A0A0A0;border-bottom:1px solid #A0A0A0;margin:10px auto;display:table;width:95%}.mw-parser-output #headerprevious,.mw-parser-output #footerprevious{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output #headernext,.mw-parser-output #footernext{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output .prev-span{float:left;display:block;font-size:smaller}.mw-parser-output .prev-span-arrow{margin-right:0.5em;position:relative;top:1.5px}.mw-parser-output .middle-div{width:68%;display:table-cell;vertical-align:middle}.mw-parser-output .next-span{float:right;display:block;font-size:smaller}.mw-parser-output .next-span-arrow{margin-left:0.5em;position:relative;top:1.5px}.mw

<a name="s-extraccion-xml"></a>

## Extracción de contenido

Como se ve al mostrar el contenido del objeto `ojson`, nos interesan los pares de clave-valor siguientes:


**Para el título**


```json
{'parse': {'title': 'Una viñeta', ...
```

**Para la autora**

```json
'links': [{'ns': 106, 'exists': '', '*': 'Autor:Delmira Agustini'}],
```

**Para el texto**
```json
'text': {'*': '<div class="mw-parser-output"> ...
```

En los casos del título y de la autora se ve claramente la localización de la información. En el caso del texto del poema, la API de Wikisource no da un campo específico con su contenido. Lo que hace es dar, como valor para la clave `text`, una cadena de texto que expresa una parte del contenido principal del cuerpo de la página en HTML (la parte central con el poema y algunos de sus metadatos). Así que debemos extraer esta cadena, crear un árbol HTML y extraer el texto del poema a partir de este. No es el uso más típico de JSON pero nos permite combinar los conocimientos vistos en el capítulo.

<a name="ss-metadatos"></a>

### Metadatos

Vamos a extraer el título y autora primero, asignándolos a una variable.

Como se ve en los ejemplos, accedemos al valor relacionado con una clave con la sintaxis `[]`, es decir `ojson["parse"]` nos da el valor asociado a la clave parse. Como ese valor es un objeto, compuesto a su vez de varios pares de clave valor, debemos dar de nuevo entre corchetes la clave para el título para acceder al valor final que nos interesa: `ojson["parse"]["title"]`.

In [3]:
titulo = ojson["parse"]["title"]

In [4]:
print(titulo)

Una viñeta


In [8]:
autora = ojson["parse"]["links"][0]["*"].replace("Autor:", "")

In [9]:
print(autora)

Delmira Agustini


<a name="ss-texto"></a>

### Texto del poema

Nos ocupamos ahora del texto. Primero extraemos la parte central del cuerpo de la página Wikisource que contiene el poema con alguna información más (como se ha dicho, la API no divide el contenido de forma más detallada).

In [10]:
contenido_cuerpo_pagina = ojson["parse"]["text"]["*"]

In [11]:
print(contenido_cuerpo_pagina)

<div class="mw-parser-output"><style data-mw-deduplicate="TemplateStyles:r1188054">.mw-parser-output #headertemplate{text-align:center;padding:1em;background:#F4F0E5;border-top:1px solid #A0A0A0;border-bottom:1px solid #A0A0A0;margin:10px auto;display:table;width:95%}.mw-parser-output #headerprevious,.mw-parser-output #footerprevious{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output #headernext,.mw-parser-output #footernext{padding:0;display:table-cell;vertical-align:middle;width:16%}.mw-parser-output .prev-span{float:left;display:block;font-size:smaller}.mw-parser-output .prev-span-arrow{margin-right:0.5em;position:relative;top:1.5px}.mw-parser-output .middle-div{width:68%;display:table-cell;vertical-align:middle}.mw-parser-output .next-span{float:right;display:block;font-size:smaller}.mw-parser-output .next-span-arrow{margin-left:0.5em;position:relative;top:1.5px}.mw-parser-output .lower-div{clear:both;padding:0.5em;text-align:center;margin:0 auto;font-si

Ya que esta cadena expresa el HTML de la página, la analizaremos con el parseador de HTML y extraeremos el texto del poema, usando las mismas operaciones que ya se vieron en el [notebook](01-extraccion-desde-html.ipynb) sobre scraping desde HTML.

In [12]:
from bs4 import BeautifulSoup

arbol_html = BeautifulSoup(contenido_cuerpo_pagina)

In [13]:
texto_bruto_poema = arbol_html.find_all("div", attrs={"class": "poem"})[0].find("p").get_text()

In [14]:
texto_bruto_poema

'\xa0Tarde sucia de invierno. El caserío,   \n\xa0como si fuera un croquis al creyón,   \n\xa0se hunde en la noche. El humo de un bohío,   \n\xa0que sube en forma de tirabuzón;   \n\xa0\n\n\xa0mancha el paisaje que produce frío,  \n\xa0y debajo de la genuflexión   \n\xa0de la arboleda, somormuja el río   \n\xa0su canción, su somnífera canción.   \n\xa0\n\n\xa0Los labradores, camellón abajo,   \n\xa0retornan fatigosos del trabajo,  \n\xa0como un problema sin definición.   \n\xa0\n\n\xa0Y el dueño del terruño, indiferente,   \n\xa0rápidamente, muy rápidamente,   \n\xa0baja en su coche por el camellón.\n'

In [15]:
versos_brutos = texto_bruto_poema.split("\n")

In [16]:
versos_brutos

['\xa0Tarde sucia de invierno. El caserío,   ',
 '\xa0como si fuera un croquis al creyón,   ',
 '\xa0se hunde en la noche. El humo de un bohío,   ',
 '\xa0que sube en forma de tirabuzón;   ',
 '\xa0',
 '',
 '\xa0mancha el paisaje que produce frío,  ',
 '\xa0y debajo de la genuflexión   ',
 '\xa0de la arboleda, somormuja el río   ',
 '\xa0su canción, su somnífera canción.   ',
 '\xa0',
 '',
 '\xa0Los labradores, camellón abajo,   ',
 '\xa0retornan fatigosos del trabajo,  ',
 '\xa0como un problema sin definición.   ',
 '\xa0',
 '',
 '\xa0Y el dueño del terruño, indiferente,   ',
 '\xa0rápidamente, muy rápidamente,   ',
 '\xa0baja en su coche por el camellón.',
 '']

In [17]:
versos_limpios = [verso.strip() for verso in versos_brutos]

In [18]:
versos_limpios

['Tarde sucia de invierno. El caserío,',
 'como si fuera un croquis al creyón,',
 'se hunde en la noche. El humo de un bohío,',
 'que sube en forma de tirabuzón;',
 '',
 '',
 'mancha el paisaje que produce frío,',
 'y debajo de la genuflexión',
 'de la arboleda, somormuja el río',
 'su canción, su somnífera canción.',
 '',
 '',
 'Los labradores, camellón abajo,',
 'retornan fatigosos del trabajo,',
 'como un problema sin definición.',
 '',
 '',
 'Y el dueño del terruño, indiferente,',
 'rápidamente, muy rápidamente,',
 'baja en su coche por el camellón.',
 '']

In [19]:
texto_poema = "\n".join(versos_limpios)

In [20]:
texto_poema

'Tarde sucia de invierno. El caserío,\ncomo si fuera un croquis al creyón,\nse hunde en la noche. El humo de un bohío,\nque sube en forma de tirabuzón;\n\n\nmancha el paisaje que produce frío,\ny debajo de la genuflexión\nde la arboleda, somormuja el río\nsu canción, su somnífera canción.\n\n\nLos labradores, camellón abajo,\nretornan fatigosos del trabajo,\ncomo un problema sin definición.\n\n\nY el dueño del terruño, indiferente,\nrápidamente, muy rápidamente,\nbaja en su coche por el camellón.\n'

In [21]:
print(texto_poema)

Tarde sucia de invierno. El caserío,
como si fuera un croquis al creyón,
se hunde en la noche. El humo de un bohío,
que sube en forma de tirabuzón;


mancha el paisaje que produce frío,
y debajo de la genuflexión
de la arboleda, somormuja el río
su canción, su somnífera canción.


Los labradores, camellón abajo,
retornan fatigosos del trabajo,
como un problema sin definición.


Y el dueño del terruño, indiferente,
rápidamente, muy rápidamente,
baja en su coche por el camellón.



<a name="s-guardar-dataframe"></a>

## Guardar la información extraída en una dataframe

Gracias a la extracción, tenemos la información que nos interesa en las variables `titulo`, `autora` y `texto_poema`.

Podemos guardar esto en una dataframe por ejemplo, como se vio en el [notebook](01-extraccion-desde-html.ipynb) sobre scraping desde HTML.

In [22]:
import pandas as pd

datos = {"autora": [autora],
         "titulo": [titulo],
         "textoPoema": [texto_poema]} 

df = pd.DataFrame(datos)

In [23]:
df

,autora,titulo,textoPoema
0,Delmira Agustini,Una viñeta,"Tarde sucia de invierno. El caserío,\ncomo si ..."


Podemos ahora guardar este archivo en el disco.

In [24]:
import pathlib

# definir directorio para el archivo de salida
directorio_actual = pathlib.Path.cwd()
dir_resultados = directorio_actual.joinpath("resultados", "desde-json")

# crear el directorio (si ya existe no será creado)
if not dir_resultados.is_dir():
    dir_resultados.mkdir(parents=True)

# preparar los nombres de los archivos de salida para cada formato
nombre_fichero = "agustini_poema"
salida_csv = dir_resultados.joinpath(nombre_fichero + ".csv")
salida_tsv = dir_resultados.joinpath(nombre_fichero + ".tsv")
salida_excel = dir_resultados.joinpath(nombre_fichero + ".xlsx")

In [25]:
df.to_csv(salida_csv, index=False)
df.to_csv(salida_tsv, sep="\t", index=False)
df.to_excel(salida_excel, index=False)

Abriendo el archivo .xlsx con LibreOffice Calc, el resultado se muestra como sigue:


![img](./img/03_json_01_excel.png)